### Extract all Books about Poetry form Project Guthenberg

In [11]:
import json
import re

import requests
from tqdm import tqdm

# for downloading the txt file directly
TXT_URL = "https://www.gutenberg.org/cache/epub/{}/pg{}.txt"


In [ ]:
poetry_books = {}
topic = "poetry"
result = requests.get(url=f"https://gutendex.com/books/?page=1&topic={topic}")
content = json.loads(result.content)

while True:
    for book in content["results"]:
        if book["languages"] == ["en"]:
            # account for different translations
            if book["title"] in poetry_books.keys():
                poetry_books[book["title"]].append(
                    TXT_URL.format(book["id"], book["id"])
                )
            else:
                poetry_books[book["title"]] = [TXT_URL.format(book["id"], book["id"])]
    print(f"Total number of books so far: {len(list(poetry_books.keys()))}")
    if not content["next"]:
        break
    result = requests.get(content["next"])
    content = json.loads(result.content)

with open("guthenberg_poetry_books.json", "w") as f:
    f.write(json.dumps(poetry_books, indent=4))


### Download all Books

In [2]:
with open("guthenberg_poetry_books.json", "r") as f:
    poetry_books = json.load(f)


In [14]:
failed_to_download = {}
for title, urls in tqdm(poetry_books.items()):
    downloaded = False
    for url in urls:
        result = requests.get(url=url)
        # always download the first available translation
        if result.status_code == 200:
            downloaded = True
            break
    if downloaded:
        clean_title = re.sub("[^A-Za-z]", "", title)
        # truncate titles that are way too long
        clean_title = clean_title[:75]
        with open(f"guthenberg_books/{clean_title}.txt", "w", encoding="utf-8") as f:
            f.write(result.content.decode("utf-8"))
    else:
        failed_to_download[title] = urls

with open("failed_to_download.json", "w") as f:
    f.write(json.dumps(failed_to_download, indent=4))

100%|██████████| 2713/2713 [34:13<00:00,  1.32it/s]
